In [1]:
import time
import os
from selenium import webdriver
import json
import csv
import tkinter as tk
from cefpython3 import cefpython as cef
import platform
import sys
import ctypes

location = str(os.getcwd())

In [2]:
work_list = []
complete_list = []
list_index = 0

#reading auto-coded file
with open(location + "/" + "database.csv", encoding = "Utf-8") as infile:
    reader = csv.DictReader(infile)
    input_list = list(reader)
for item in input_list:
    if int(item["to_code"]) == 1:
        work_list.append(item)
    else:
        complete_list.append(item)

#reading auto-coded file       
if os.path.exists(location + '/to_code.csv') == True:
    with open(location + "/" + "to_code.csv", encoding = "Utf-8") as infile:
        reader = csv.DictReader(infile)
        input_list = list(reader)
    for item in input_list:
        if int(item["to_code"]) == 1:
            work_list.append(item)
        else:
            complete_list.append(item)

#REDO THIS
name = (work_list[list_index]['Given_fa']+" "+str(work_list[list_index]['Family_fa']))
title = (work_list[list_index]["Paper_articleTitle"])
institution = (work_list[list_index]["fa_insti"].split(",")[0])
date = (work_list[list_index]['Paper_publicationYear'])

In [3]:
#Initializing and Exiting
def initialize_browser_and_csv():
    global name
    global title
    global institution
    global driver
    global date
    global list_index
    if not list_index:
        list_index = 0
    #adding browser frame
#     driver = None
#     if not driver:
#         options=webdriver.ChromeOptions()
#         try:
#             driver = webdriver.Chrome(location + "/chromedriver", options=options)
#             url = 'https://google.com/'
#             driver.get(url)
            
#         except Exception as error:
#             error_string = str(error)
#             error_string = error_string.split("created:")[1]
#             error_string1 = "Current " + error_string.split("Current")[1]
#             error_string1 = error_string1.split("with")[0]
#             error_string = error_string.split("Current")[0]
            
#             link_text = "Please download correct version of ChromeDriver here: https://chromedriver.chromium.org/downloads"
            
#             if name:
#                 lname.delete(0,1000)
#                 lname.insert(-1, error_string)
#             if title:
#                 ltitle.delete(0,1000)
#                 ltitle.insert(-1, link_text)
#             if institution:
#                 linstitution.delete(0,1000)
#                 linstitution.insert(-1, error_string1)

            
def exit_all():
    global driver
    global list_index
    global complete_list
    if list_index:
        list_index = None
    try:
        driver.close()
    except:
        pass

#writing a file that has all completed authors
    if complete_list:
        fields = complete_list[0].keys()
        filename = "complete.csv"
        if os.path.exists(location + '/complete.csv') == True:
            try:
                os.remove('complete.csv')
            except:
                filename = "coding_output_overflow.csv"
        
        output_rows = []
        for author in complete_list:
            row = []
            for item in fields:
                row.append(author[item])
            output_rows.append(row)
        with open(filename, 'w', newline='', encoding = "Utf-8") as csvfile: 
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(fields) 
            csvwriter.writerows(output_rows)
            
#writing a file that has the authors that need to be coded    
    if work_list:
        fields = work_list[0].keys()
        filename = "to_code.csv"
        if os.path.exists(location + '/to_code.csv') == True:
            try:
                os.remove('to_code.csv')
            except:
                filename = "coding_input_overflow.csv"
        
        output_rows = []
        for author in work_list:
            row = []
            for item in fields:
                row.append(author[item])
            output_rows.append(row)
        with open(filename, 'w', newline='', encoding = "Utf-8") as csvfile: 
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(fields) 
            csvwriter.writerows(output_rows)
            
    #deleting the auto_coded file as it is redundant
    if os.path.exists(location + '/database.csv') == True:
        os.remove('database.csv')

#reults
def search_and_grab():
    global driver
    name = nvar.get()
    title = tvar.get()
    institution = ivar.get()
    search_string = ""
    
    search_method = search_method_var.get()
    if search_method == 1:
        search_string += ("https://scholar.google.com/scholar?hl=en&as_sdt=0%2C15&q=")
    elif search_method == 0:
        search_string += ("https://www.researchgate.net/search.Search.html?type=researcher&query=")
    else:
        search_string += ("https://www.google.com/search?q=")    
    search_string += (name)
    
    use_institute = institution_check_var.get()
    use_title = title_check_var.get()
    if use_institute == 1:
        search_string += ("+" + institution)
    if use_title == 1:
        search_string += ("+" + title)

    
    name = name
    driver.get(search_string)

    
#Moving through authors
def next_author():
    global name
    global title
    global institution
    global date
    global list_index
        
    try:
        if name:
            name = (work_list[list_index]['Given_fa']+" "+str(work_list[list_index]['Family_fa']))
            lname.delete(0,1000)
            lname.insert(-1, name)
        if title:
            title = (work_list[list_index]["Paper_articleTitle"])
            ltitle.delete(0,1000)
            ltitle.insert(-1, title)
        if institution:
            institution = (work_list[list_index]["fa_insti"].split(",")[0])
            linstitution.delete(0,1000)
            linstitution.insert(-1,institution)
        if date:
            date = (work_list[list_index]['Paper_publicationYear'][0:-1])
            date_label.config(text=("Year Published: " + date))
    except Exception as e:
        print(e)
        lname.delete(0,1000)
        ltitle.delete(0,1000)
        linstitution.delete(0,1000)
        lname.insert(-1, "List Exhausted: Please Click Exit")
        ltitle.insert(-1, '')
        linstitution.insert(-1, '')

In [4]:
def submit_coding():
    name = nvar.get()
    title = tvar.get()
    institution = ivar.get()
    global work_list
    global complete_list
    global list_index
    race = code_var.get()
    gender = gender_var.get()
    
    if race == 0:
        race_value = "Non-White"
    elif race == 1:
        race_value = "White"
    elif race == 2:
        race_value = "N/A"
        
    if gender == 0:
        gender_value = "Female"
    elif gender == 1:
        gender_value = "Male"
    elif gender == 2:
        gender_value = "Non-Binary"
    elif gender == 3:
        gender_value = "N/A"
    
    work_list[list_index]["to_code"] = 0
    work_list[list_index]["gender_code"] = gender_value
    work_list[list_index]["race_code"] = race_value
    work_list[list_index]["to_code"] = 0
    complete_list.append(work_list[list_index])
    del work_list[list_index]
    next_author()

In [5]:
# platforms
####
#Browser frame handling from: https://stackoverflow.com/questions/69013867/load-cef-in-labelframe
####
WINDOWS = platform.system() == 'Windows'
LINUX = platform.system() == 'Linux'
MAC = platform.system() == 'Darwin'

class BrowserFrame(tk.Frame):
    def __init__(self, master=None, **kw):
        super().__init__(master, **kw)
        self.browser = None
        self.bind('<Configure>', self.on_configure)

    def get_window_handle(self):
        if MAC:
            from AppKit import NSApp
            import objc
            return objc.pyobjc_id(NSApp.windows()[-1].contentView())
        elif self.winfo_id() > 0:
            return self.winfo_id()
        else:
            raise Exception('Could not obtain window handle!')

    def on_configure(self, event):
        if self.browser is None:
            # create the browser and embed it in current frame
            rect = [0, 0, self.winfo_width(), self.winfo_height()]
            cef_winfo = cef.WindowInfo()
            win_id = self.get_window_handle()
            cef_winfo.SetAsChild(win_id, rect)
            
            
            self.browser = cef.CreateBrowserSync(cef_winfo, url="https://www.google.com/")
            # start the browser handling loop
            self.cef_loop()
            ctypes.windll.user32.SetWindowPos(
            self.browser.GetWindowHandle(), 0,
            0, 0, event.width, event.height, 0x0002)
    def cef_loop(self):
        cef.MessageLoopWork()
        self.after(10, self.cef_loop)

In [6]:
#GUI
def main():
    window = tk.Tk()
    window.title("WilCite")
    institution_check_var = tk.IntVar()
    title_check_var = tk.IntVar()
    search_method_var = tk.IntVar()
    code_var = tk.IntVar()
    gender_var = tk.IntVar()
    nvar = tk.StringVar()
    ivar = tk.StringVar()
    tvar = tk.StringVar()

    #info
    name_label = tk.Label(text=("Name: "), font=("Helvetica", 12))
    name_label.grid(row = 0, column = 0, sticky = "W")
    lname = tk.Entry(textvariable = nvar, width = 50)
    lname.grid(row = 0, column = 1, sticky = "W")
    lname.insert(-1, name)

    institution_label = tk.Label(text=("Institution: "), font=("Helvetica", 12))
    institution_label.grid(row = 1, column = 0, sticky = "W")
    linstitution = tk.Entry(textvariable = ivar, width = 150)
    linstitution.grid(row=1, column=1, columnspan = 2, sticky = "W")
    linstitution.insert(-1, institution)

    title_label = tk.Label(text=("Paper Title: "), font=("Helvetica", 12))
    title_label.grid(row = 2, column = 0, sticky = "W")
    ltitle = tk.Entry(textvariable = tvar, width = 150)
    ltitle.grid(row = 2, column = 1, columnspan = 2, sticky = "W")
    ltitle.insert(-1, title)

    date_label = tk.Label(text=("Year Published: "), font=("Helvetica", 12))
    date_label.grid(row = 0, column = 2, sticky = "W")
    date_label.config(text=("Year Published: " + date))

    #Initialize and exit
    start_button = tk.Button(text="Initialize", command = initialize_browser_and_csv, font=("Helvetica", 10))
    start_button.grid(row = 0, column = 3)

    exit_button = tk.Button(text="Quit/Compile", command=exit_all, font=("Helvetica", 10))
    exit_button.grid(row = 1, column = 3)

    #Stylistic Break
    break_label = tk.Label(text=("====="), font=("Helvetica", 12))
    break_label.grid(row=3, column=0)
    break_label1 = tk.Label(text=("====="), font=("Helvetica", 12))
    break_label1.grid(row=3, column=1)
    break_label2 = tk.Label(text=("====="), font=("Helvetica", 12))
    break_label2.grid(row=3, column=2)
    break_label3 = tk.Label(text=("====="), font=("Helvetica", 12))
    break_label3.grid(row=3, column=3)

    #search terms
    info_label = tk.Label(text=("Search Terms:"), font=("Helvetica", 12))
    info_label.grid(row = 4, column = 1)

    checkbox_1 = tk.Checkbutton(text="Title", variable=title_check_var)
    checkbox_1.grid(row = 6, column = 1)

    checkbox_2 = tk.Checkbutton(text="Institution", variable=institution_check_var)
    checkbox_2.grid(row=5, column = 1)

    #Search_method
    method_label = tk.Label(text=("Search Method:"), font=("Helvetica", 12))
    method_label.grid(row = 4, column = 0)

    google_radio = tk.Radiobutton(text="Google", variable=search_method_var, value=2)
    google_radio.grid(row=5, column = 0, sticky = "W")

    research_gate_radio = tk.Radiobutton(text="Research Gate", variable=search_method_var, value=0)
    research_gate_radio.grid(row=6, column = 0, sticky = "W")

    google_scholar_radio = tk.Radiobutton(text="Google Scholar", variable=search_method_var, value=1)
    google_scholar_radio.grid(row=7, column = 0, sticky = "W")

    #go
    search_button = tk.Button(text="Search", command = search_and_grab, font=("Helvetica", 10))
    search_button.grid(row = 7, column = 1)

    #BIPOC Coding
    bipoc_label = tk.Label(text=("BIPOC Coding:"), font=("Helvetica", 12))
    bipoc_label.grid(row = 4, column = 3)

    bipoc = tk.Radiobutton(text="Mark as BIPOC", variable = code_var, value = 0, font=("Helvetica", 10))
    bipoc.grid(row = 5, column = 3)

    not_bipoc = tk.Radiobutton(text="Mark as not BIPOC", variable = code_var, value = 1, font=("Helvetica", 10))
    not_bipoc.grid(row = 6, column = 3)

    unknown_button = tk.Radiobutton(text="Cannot Determine", variable = code_var, value = 2, font=("Helvetica", 10))
    unknown_button.grid(row = 7, column = 3)

    #gender coding
    gender_label = tk.Label(text=("Gender Coding:"), font=("Helvetica", 12))
    gender_label.grid(row = 4, column = 2)

    female_radio = tk.Radiobutton(text="Woman", variable = gender_var, value = 0, font=("Helvetica", 10))
    female_radio.grid(row = 5, column = 2)

    male_radio = tk.Radiobutton(text="Man", variable = gender_var, value = 1, font=("Helvetica", 10))
    male_radio.grid(row = 6, column = 2)

    nonbinary_radio = tk.Radiobutton(text="Non-Binary", variable = gender_var, value = 2, font=("Helvetica", 10))
    nonbinary_radio.grid(row = 7, column = 2)

    unknown_gender_radio = tk.Radiobutton(text="Cannot Determine", variable = gender_var, value = 3, font=("Helvetica", 10))
    unknown_gender_radio.grid(row = 8, column = 2)

    submit = tk.Button(text = "Submit", command = submit_coding, font=("Helvetica", 10))
    submit.grid(row = 8, column = 3)
    
    cef.Initialize()
    browser = BrowserFrame(window, height = 700, width = 1300)
    browser.grid(column = 0, row = 9, sticky = "W", padx=30, columnspan = 4)
    
    window.mainloop()
    cef.Shutdown()

In [7]:
main()

In [10]:
# Make "undo" button
# add "other" option for bipoc and gender
# add notes field